## Implementation of CNN from SCRATCH

In [35]:
import numpy as np
import math

In [37]:
class Conv2d:
    id_ = 0
    def __init__(self,input_x:np.ndarray,filter_shape:tuple,number_of_filters:int,padding:int=1,
                 stride:int=1,max_pool_shape:tuple=None,max_pool_stride:int=2):
        self.input_x = input_x # (channel,H,W) # X
        self.filter_shape = filter_shape # (F_h,F_W)
        self.number_of_filters = number_of_filters
        self.padding = padding
        self.stride = stride
        self.filters = None # W
        self.max_pool_shape = max_pool_shape
        self.max_pool_stride = max_pool_stride        
        self.id_ = Conv2d.id_ 
        Conv2d.id_ += 1
        
    def xavier_weight_init(self,shape:tuple,fan_in:int,fan_out:int) -> np.ndarray:
        return np.random.randn(*shape)*np.sqrt(2/(fan_in+fan_out))
        
    # forward
    def __call__(self,is_predicting=False) -> np.ndarray:
        
        # Calculating output Dimensions
        output_h = math.floor((((self.input_x.shape[1]+2*self.padding-self.filter_shape[0]) /self.stride) + 1 ))
        output_w = math.floor((((self.input_x.shape[2]+2*self.padding-self.filter_shape[1]) /self.stride) + 1 ))
        
        
        # Padding
        if self.padding != 0:
            ch_list = [np.pad(self.input_x[x],self.padding) for x in range(self.input_x.shape[0])]
            self.input_x = np.vstack([[ch] for ch in ch_list])
        

        # Initializing output
        output = np.zeros((self.number_of_filters,output_h,output_w)) # (number_of_filters,H,W)
        
        # Initializing filters
        if not is_predicting:
            self.filters = [self.xavier_weight_init(
                shape=(self.input_x.shape[0],self.filter_shape[0],self.filter_shape[1]),
                fan_in = self.input_x.shape[0] * self.input_x.shape[1] * self.input_x.shape[2],
                fan_out = self.filter_shape[0] * self.filter_shape[1] * self.input_x.shape[0] if self.max_pool_shape is None else self.filter_shape[0] * self.filter_shape[1] * self.input_x.shape[0] /(self.max_pool_shape[0]*self.max_pool_shape[1])  
            ) for _ in range(self.number_of_filters)]

        # Convolving
        for index, filter_ in enumerate(self.filters):
            out = np.empty((output_h,output_w))
            row_counter = 0
            for row in np.arange(0,self.input_x.shape[1],step=self.stride):
                col_counter = 0
                if row+self.filter_shape[0] > self.input_x.shape[1] or row > self.input_x.shape[1]:
                    pass
                else:
                    for col in np.arange(0,self.input_x.shape[2],step=self.stride):
                        if col+self.filter_shape[1] >  self.input_x.shape[2] or col > self.input_x.shape[2]:
                            pass
                        else:
                            out[row_counter,col_counter] =(self.input_x[:,row:row+self.filter_shape[0],col:col+self.filter_shape[1]] * filter_).sum()
                            col_counter += 1
                    
                    row_counter += 1
            output[index] = out
        
        cache = (self.input_x,)
        
        # Max Pooling       
        if self.max_pool_shape is not None:
            pooling_output = np.zeros((output.shape[0],
                                       ((output.shape[1]-self.max_pool_shape[0])//self.max_pool_stride)+1,
                                       ((output.shape[2]-self.max_pool_shape[1])//self.max_pool_stride)+1))
            for i in range(output.shape[0]):
                row_counter = 0
                for row in np.arange(0,self.input_x.shape[1],step=self.max_pool_stride):
                    col_counter = 0
                    if row+self.max_pool_shape[0] > output.shape[1]:
                        pass
                    else:
                        for col in np.arange(0,self.input_x.shape[2],step=self.max_pool_stride):
                            if col+self.max_pool_shape[1] > output.shape[2]:
                                pass
                            else:
                                pooling_output[i,row_counter,col_counter] = output[i,row:row+self.max_pool_shape[0],col:col+self.max_pool_shape[1]].max()
                                col_counter += 1 
                        row_counter += 1
       
            return pooling_output
        
        return output  
    
    def backward(self,dh):
        print(self.input_x.shape)
        dx = np.zeros(self.input_x.shape)
        dw = np.zeros(self.input_x.shape[0],self.filter_shape[0],self.filter_shape[1])
        
        (n_C,n_H, n_W) = dH.shape
#         (f, f) = self.filters.shape

        for ch in range(n_C):
            for h in range(n_H):
                for w in range(n_W):
                    dX[ch,h:h+f, w:w+f] += W * dH[ch,h,w]
                    dW += X[ch,h:h+f, w:w+f] * dH[ch,h,w]

        return dX, dW
        
        
        

In [38]:
A = np.arange(225).reshape((9,5,5))

In [39]:
A.shape

(9, 5, 5)

In [40]:
conv2d = Conv2d(input_x=A,filter_shape=(1,1),number_of_filters=1,padding=1,stride=2)

In [41]:
y = conv2d(is_predicting=False)

In [42]:
y.shape

(1, 4, 4)

In [43]:
conv2d_2 = Conv2d(input_x=y,filter_shape=(2,2),number_of_filters=3,padding=2,stride=2)

In [44]:
yy = conv2d_2()
yy.shape

(3, 4, 4)

In [50]:
conv2d.backward(2)

(9, 7, 7)
